In [1]:
import os
import pandas as pd
import time
import glob
from IPython.display import clear_output


from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_f
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, datediff, floor, col, avg, substring, when, length, lpad, monotonically_increasing_id, expr, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from concurrent.futures import ThreadPoolExecutor

spark = SparkSession.builder.getOrCreate()

In [2]:
#Function to measure execution time...
def time_execution(task_name, func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    duration = end_time - start_time
    print(f"{task_name} executed in {duration:.2f} seconds")
    return result, duration


## 1) Creating the spark dataframes 

In [3]:
#setting path to HDFS folder
path = '/synthea_output/'


In [5]:
#Patient files
observations = spark.read.csv(path+"observations.csv", header=True)
patient = spark.read.csv(path+"patients.csv", header=True) 
#Medical files
careplans = spark.read.csv(path+"careplans.csv", header=True)
conditions = spark.read.csv(path+"conditions.csv", header=True)
procedures=spark.read.csv(path+"procedures.csv", header=True)
encounters = spark.read.csv(path+"encounters.csv", header=True)
medications = spark.read.csv(path+"medications.csv", header=True)
#Insurance and hospital files
payer_transitions=spark.read.csv(path+"payer_transitions.csv", header=True)
payers=spark.read.csv(path+"payers.csv", header=True)
providers=spark.read.csv(path+"providers.csv", header=True)
organizations=spark.read.csv(path+"organizations.csv", header=True)

## 3) Cleaning dataframes and renaming variables 

In [6]:
#Renaming columns
patient = (
    patient.withColumnRenamed("Id", "patient_id")
           .withColumnRenamed("MARITAL", "patient_marital")
           .withColumnRenamed("RACE", "patient_race")
           .withColumnRenamed("ETHNICITY", "patient_ethnicity")
           .withColumnRenamed("GENDER", "patient_gender")
           .withColumnRenamed("ZIP", "patient_zip")
)
encounters = (
    encounters.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("Id", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "encounter_discription")
              .withColumnRenamed("CODE", "encounter_code")
              .withColumnRenamed("START", "encounter_start")
              .withColumn("encounter_start", to_date("encounter_start"))
              .withColumnRenamed("STOP", "encounter_stop")
              .withColumn("encounter_stop", to_date("encounter_stop"))
              .withColumn("PATIENT COST", col("TOTAL_CLAIM_COST") - col("PAYER_COVERAGE"))
              .withColumnRenamed("PAYER", "payer_id")
              .withColumnRenamed("ORGANIZATION", "organization_id")
              .withColumnRenamed("PROVIDER", "provider_id")
)
careplans = (
    careplans.withColumnRenamed("PATIENT", "patient_id")
             .withColumnRenamed("Id", "careplan_id")
             .withColumnRenamed("ENCOUNTER", "encounter_id")
             .withColumnRenamed("DESCRIPTION", "careplan_descriptions")
             .withColumnRenamed("CODE", "careplan_code")
)
procedures = (
    procedures.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("ENCOUNTER", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "procedure_descriptions")
              .withColumnRenamed("CODE", "procedure_code")
              .withColumnRenamed("DATE", "procedure_date")
              .withColumnRenamed("BASE_COST", "procedure_cost")
)
conditions = (
    conditions.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("ENCOUNTER", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "condition_description")
              .withColumnRenamed("CODE", "condition_code")
              .withColumnRenamed("START", "condition_start")
              .withColumnRenamed("END", "condition_end")
)
observations = (
    observations.withColumnRenamed("PATIENT", "patient_id")
                .withColumnRenamed("ENCOUNTER", "encounter_id")
                .withColumnRenamed("DATE", "observation_date")
                .withColumn("observation_date", to_date("observation_date"))
                .withColumn("obs_value", col("VALUE").cast("double"))
                .withColumnRenamed("CODE", "observation_code")
                .withColumnRenamed("DESCRIPTION", "observation_description")
)
medications = (
    medications.withColumnRenamed("START", "medication_start")
               .withColumn("medication_start", to_date("medication_start"))
               .withColumnRenamed("STOP", "medication_stop")
               .withColumn("medication_stop", to_date("medication_stop"))
               .withColumnRenamed("PATIENT", "patient_id")
               .withColumnRenamed("PAYER", "payer_id")
               .withColumnRenamed("ENCOUNTER", "encounter_id")
               .withColumnRenamed("CODE", "medication_code")
               .withColumnRenamed("DESCRIPTION", "medication_description")
)
payer_transitions = (
    payer_transitions.withColumnRenamed("PATIENT", "patient_id")
                     .withColumnRenamed("PAYER", "payer_id")
)
payers = (
    payers.withColumnRenamed("Id", "payer_id")
          .withColumnRenamed("NAME", "payer_name")
          .withColumnRenamed("OWNERSHIP", "payer_ownership")
)
providers = (
    providers.withColumnRenamed("Id", "provider_id")
             .withColumnRenamed("SPECIALITY", "provider_specialty")
)
organizations = (
    organizations.withColumnRenamed("Id", "organization_id")
                 .withColumnRenamed("NAME", "organization_name")
                 .withColumnRenamed("ZIP", "organization_zip")
)
organizations = organizations.withColumn(
    "organization_zip",
    col("organization_zip").cast("string")
)

#adding leading 0's to zip codes to retain their information
organizations = organizations.withColumn(
    "organization_zip",
    when(length(col("organization_zip")) == 4,  
    lpad(col("organization_zip"), 5, "0")
).otherwise(col("organization_zip")))   

In [7]:
#Merge together dataframes on various id fields that will be used for ML modeling...
encounters = (
    encounters
    .join(payers.select("payer_id", "payer_name", "payer_ownership"), on="payer_id", how="left")
    .join(organizations.select("organization_id", "organization_name", "organization_zip"), on="organization_id", how="left")
    .join(providers.select("provider_id", "provider_specialty"), on="provider_id", how="left")
    .join(procedures.select("encounter_id", "procedure_descriptions", "procedure_code"), on="encounter_id", how="left")
    .join(patient.select("patient_id", "BIRTHDATE", "patient_marital", "patient_race", "patient_ethnicity", "patient_gender", "patient_zip"), on="patient_id", how="left")
    .withColumn("age_at_encounter", floor(datediff(col("encounter_start"), col("BIRTHDATE")) / 365.25))
)


In [7]:
encounters.show(5)

25/05/06 13:04:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------+--------------+--------------+---------------------+-------------------+----------------+--------------+----------+-----------------+-----------------+----------+---------------+-----------------+----------------+------------------+----------------------+--------------+----------+---------------+------------+-----------------+--------------+-----------+----------------+
|          patient_id|        encounter_id|         provider_id|     organization_id|            payer_id|encounter_start|encounter_stop|ENCOUNTERCLASS|encounter_code|encounter_discription|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|REASONCODE|REASONDESCRIPTION|     PATIENT COST|payer_name|payer_ownership|organization_name|organization_zip|provider_specialty|procedure_descriptions|procedure_code| BIRTHDATE|patient_marital|patient_race|patient_ethnicity|patient_gender|patient_zi

In [8]:
encounters.columns

['patient_id',
 'encounter_id',
 'provider_id',
 'organization_id',
 'payer_id',
 'encounter_start',
 'encounter_stop',
 'ENCOUNTERCLASS',
 'encounter_code',
 'encounter_discription',
 'BASE_ENCOUNTER_COST',
 'TOTAL_CLAIM_COST',
 'PAYER_COVERAGE',
 'REASONCODE',
 'REASONDESCRIPTION',
 'PATIENT COST',
 'payer_name',
 'payer_ownership',
 'organization_name',
 'organization_zip',
 'provider_specialty',
 'procedure_descriptions',
 'procedure_code',
 'BIRTHDATE',
 'patient_marital',
 'patient_race',
 'patient_ethnicity',
 'patient_gender',
 'patient_zip',
 'age_at_encounter']

In [9]:
encounters.count()

11596

## Preparing Data

In [8]:
modeling_df = encounters.select(
    col("PATIENT COST").cast("double").alias("label"),
    col("age_at_encounter").cast("double"),
    col("patient_marital"),
    col("patient_race"),
    col("patient_ethnicity"),
    col("patient_gender"),
    col("ENCOUNTERCLASS"),
    col("payer_ownership"),
    col("payer_name"),
    col("organization_zip"),
    col("organization_name"),
    col("procedure_code"),
    #col("encounter_discription"), # don't use for RF, LR and GBT models
    col("encounter_code"),
    #col("REASONDESCRIPTION"), # don't use for RF, LR and GBT models
).na.drop().filter(col("PATIENT COST") != 0)


#define categorical and numeric columns
categorical_cols = ['patient_marital', 'patient_race', 'patient_ethnicity', 
                   'patient_gender', 'ENCOUNTERCLASS',
                   'payer_ownership',"payer_name","organization_name", "organization_zip", 'procedure_code',"encounter_code"]
numeric_cols = ['age_at_encounter']

target_col = "label"

#create feature engineering pipeline stages
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") 
            for col in categorical_cols]

encoder = OneHotEncoder(
    inputCols=[col+"_index" for col in categorical_cols],
    outputCols=[col+"_encoded" for col in categorical_cols],
    dropLast=True
)

assembler = VectorAssembler(
    inputCols=numeric_cols + [col+"_encoded" for col in categorical_cols],
    outputCol="features"
)

#cache after feature engineering
feature_pipeline = Pipeline(stages=indexers + [encoder, assembler])
feature_model = feature_pipeline.fit(modeling_df)
feature_df = feature_model.transform(modeling_df).select("features", target_col).cache()

In [9]:
#Splitting training and test dataframes nd present the number of partitions of the training dataframe
train_df, test_df = time_execution(
    "Data splitting",
    lambda: feature_df.randomSplit([0.8, 0.2], seed=53)
)[0]
train_df.rdd.getNumPartitions()

Data splitting executed in 0.04 seconds


1

In [10]:
#repartition data to distribute it across the cluster
train_df = train_df.repartition(12)
print(f"Training Data Partitions: {train_df.rdd.getNumPartitions()}")

Training Data Partitions: 12


## Random Forest Model

In [23]:
#Random Forest
#common evaluator for regression
reg_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction")

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="label",
    numTrees=30,
    maxDepth=10,
    subsamplingRate=0.7,
    featureSubsetStrategy='sqrt',
    seed=42
)

rf_model, rf_time = time_execution(
    "Random Forest training",
    lambda: rf.fit(train_df)
)

rf_predictions = rf_model.transform(test_df)


print("Random Forest Regression Results:")
print("RMSE:", reg_evaluator.evaluate(rf_predictions, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(rf_predictions, {reg_evaluator.metricName: "r2"}))

Random Forest training executed in 9.74 seconds
Random Forest Regression Results:
RMSE: 1477.6584881206581
R2: 0.7758500377849268


In [24]:
rf_predictions.show(20)

+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|(383,[0,1,5,8,10,...|            159.11|1439.6557229153575|
|(383,[0,1,5,8,10,...|            366.88|1439.6557229153575|
|(383,[0,1,5,8,10,...| 288.8900000000001|1432.3595797054809|
|(383,[0,1,5,8,10,...| 621.0699999999997| 1117.164496470904|
|(383,[0,1,5,8,10,...| 707.3499999999999|1132.9762016738578|
|(383,[0,1,5,8,10,...| 621.0699999999997|1132.9762016738578|
|(383,[0,1,5,8,10,...| 879.9100000000003|2088.6252415867643|
|(383,[0,1,5,8,10,...| 966.1899999999996| 2118.173553336014|
|(383,[0,1,5,8,10,...| 879.9100000000003| 2113.322287846472|
|(383,[0,1,5,8,10,...| 879.9100000000003| 2113.322287846472|
|(383,[0,1,5,8,10,...| 966.1899999999996| 2113.322287846472|
|(383,[0,1,5,8,10,...| 707.3499999999999| 2113.322287846472|
|(383,[0,1,5,8,10,...| 879.9100000000003|1971.6798257524542|
|(383,[0,1,5,8,10,...| 8

## Linear Regression Model

In [25]:
#Linear Regression
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    regParam=0.01,
    elasticNetParam=0.5
)

lr_model, lr_time = time_execution(
    "Linear Regression training",
    lambda: lr.fit(train_df)
)

lr_predictions = lr_model.transform(test_df)

print("\nLinear Regression Results:")
print("RMSE:", reg_evaluator.evaluate(lr_predictions, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(lr_predictions, {reg_evaluator.metricName: "r2"}))


Linear Regression training executed in 1.23 seconds

Linear Regression Results:
RMSE: 1434.0246993719204
R2: 0.7888924393529015


In [26]:
lr_predictions.show(20)

+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|(383,[0,1,5,8,10,...|            159.11|1036.4825625599585|
|(383,[0,1,5,8,10,...|            366.88|1083.8866571009191|
|(383,[0,1,5,8,10,...| 288.8900000000001|-463.2288117465964|
|(383,[0,1,5,8,10,...| 621.0699999999997| 948.8139338865722|
|(383,[0,1,5,8,10,...| 707.3499999999999| 1015.860293613229|
|(383,[0,1,5,8,10,...| 621.0699999999997| 918.0783351422444|
|(383,[0,1,5,8,10,...| 879.9100000000003| 1502.778192964232|
|(383,[0,1,5,8,10,...| 966.1899999999996|1494.1438364621804|
|(383,[0,1,5,8,10,...| 879.9100000000003|1452.4497582162387|
|(383,[0,1,5,8,10,...| 879.9100000000003|1476.1518054867195|
|(383,[0,1,5,8,10,...| 966.1899999999996|1499.8538527571993|
|(383,[0,1,5,8,10,...| 707.3499999999999|1570.9599945686412|
|(383,[0,1,5,8,10,...| 879.9100000000003| 1432.743975182951|
|(383,[0,1,5,8,10,...| 8

## Gradient Boosted Tree Model

In [27]:
#Gradient Boosted Trees Regressor
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="label",
    maxIter=20,
    maxDepth=10,
    stepSize=0.1,
    subsamplingRate=0.7,
    seed=42
)

gbt_model, gbt_time = time_execution(
    "GBT training",
    lambda: gbt.fit(train_df)
)

gbt_predictions = gbt_model.transform(test_df)

print("\nGBT Regression Results:")
print("RMSE:", reg_evaluator.evaluate(gbt_predictions, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(gbt_predictions, {reg_evaluator.metricName: "r2"}))

GBT training executed in 79.01 seconds

GBT Regression Results:
RMSE: 650.1911985820497
R2: 0.956601702073667


In [28]:
gbt_predictions.show(20)

+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|(383,[0,1,5,8,10,...|            159.11|400.98862699654615|
|(383,[0,1,5,8,10,...|            366.88|413.30548873598667|
|(383,[0,1,5,8,10,...| 288.8900000000001| 337.0302477184239|
|(383,[0,1,5,8,10,...| 621.0699999999997| 662.4052830897476|
|(383,[0,1,5,8,10,...| 707.3499999999999| 662.4052830897476|
|(383,[0,1,5,8,10,...| 621.0699999999997| 662.4052830897476|
|(383,[0,1,5,8,10,...| 879.9100000000003| 636.8609859965644|
|(383,[0,1,5,8,10,...| 966.1899999999996| 636.8609859965644|
|(383,[0,1,5,8,10,...| 879.9100000000003| 648.6748895223498|
|(383,[0,1,5,8,10,...| 879.9100000000003| 636.8609859965644|
|(383,[0,1,5,8,10,...| 966.1899999999996| 636.8609859965644|
|(383,[0,1,5,8,10,...| 707.3499999999999| 593.1086145219252|
|(383,[0,1,5,8,10,...| 879.9100000000003| 636.8609859965644|
|(383,[0,1,5,8,10,...| 8

## Ensemble of ML Models 

In [11]:
#total execution timer
total_start = time.time()

#split training data into 3 parts for the 3 models
split_start = time.time()
rf_df, lr_df, gbt_df = train_df.randomSplit([1.0, 1.0, 1.0], seed=42)
split_time = time.time() - split_start
print(f"Data splitting completed in {split_time:.2f} seconds")

#define model training functions
def train_rf():
    start = time.time()
    rf = RandomForestRegressor(featuresCol="features", labelCol="label", numTrees=30, maxDepth=10, seed=42)
    model = rf.fit(rf_df)
    print(f"Random Forest trained in {time.time() - start:.2f} seconds")
    return model

def train_lr():
    start = time.time()
    lr = LinearRegression(featuresCol="features", labelCol="label", regParam=0.01, elasticNetParam=0.5)
    model = lr.fit(lr_df)
    print(f"Linear Regression trained in {time.time() - start:.2f} seconds")
    return model

def train_gbt():
    start = time.time()
    gbt = GBTRegressor(featuresCol="features", labelCol="label", maxIter=20, maxDepth=10, stepSize=0.1, seed=42)
    model = gbt.fit(gbt_df)
    print(f"GBT trained in {time.time() - start:.2f} seconds")
    return model

#train models in parallel
train_start = time.time()
with ThreadPoolExecutor() as executor:
    rf_future = executor.submit(train_rf)
    lr_future = executor.submit(train_lr)
    gbt_future = executor.submit(train_gbt)

    rf_model = rf_future.result()
    lr_model = lr_future.result()
    gbt_model = gbt_future.result()
train_time = time.time() - train_start
print(f"Total model training time (parallel): {train_time:.2f} seconds")

#make predictions and add row indices
pred_start = time.time()
rf_pred = rf_model.transform(test_df).select("prediction", "label") \
    .withColumnRenamed("prediction", "rf_pred") \
    .withColumn("row_idx", monotonically_increasing_id())

lr_pred = lr_model.transform(test_df).select("prediction") \
    .withColumnRenamed("prediction", "lr_pred") \
    .withColumn("row_idx", monotonically_increasing_id())

gbt_pred = gbt_model.transform(test_df).select("prediction") \
    .withColumnRenamed("prediction", "gbt_pred") \
    .withColumn("row_idx", monotonically_increasing_id())
pred_time = time.time() - pred_start
print(f"Prediction generation completed in {pred_time:.2f} seconds")

#join on row index
join_start = time.time()
ensemble_df = rf_pred \
    .join(lr_pred, on="row_idx") \
    .join(gbt_pred, on="row_idx") \
    .withColumn("ensemble_prediction", expr("(rf_pred + lr_pred + gbt_pred)/3"))
join_time = time.time() - join_start
print(f"Prediction joining completed in {join_time:.2f} seconds")

#evaluate ensemble
eval_start = time.time()
reg_evaluator = RegressionEvaluator(labelCol="label", predictionCol="ensemble_prediction")

print("\nEnsemble Results:")
print("RMSE:", reg_evaluator.evaluate(ensemble_df, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(ensemble_df, {reg_evaluator.metricName: "r2"}))
eval_time = time.time() - eval_start
print(f"Evaluation completed in {eval_time:.2f} seconds")

#execution time
total_time = time.time() - total_start
print(f"\nTotal execution time: {total_time:.2f} seconds")

Data splitting completed in 0.03 seconds


Linear Regression trained in 17.93 seconds


25/05/06 13:28:59 WARN DAGScheduler: Broadcasting large task binary with size 1117.5 KiB


Random Forest trained in 50.32 seconds


GBT trained in 162.45 seconds
Total model training time (parallel): 162.45 seconds
Prediction generation completed in 0.40 seconds
Prediction joining completed in 0.40 seconds

Ensemble Results:
RMSE: 952.0793265473858
R2: 0.9069456254853246
Evaluation completed in 1.89 seconds

Total execution time: 165.18 seconds


In [22]:
ensemble_df.show(20)

+-------+------------------+------------------+------------------+------------------+-------------------+
|row_idx|           rf_pred|             label|           lr_pred|          gbt_pred|ensemble_prediction|
+-------+------------------+------------------+------------------+------------------+-------------------+
|      0| 467.0617848174532|            159.11|1480.1516563757305|481.48461720255415|   809.566019465246|
|      1|   459.09445148412|            366.88| 1510.296136963575| 476.9364500142359|  815.4423461539769|
|      2| 567.0366807539612| 288.8900000000001|-978.8062214013314|311.65928812988875| -33.37008417249384|
|      3| 581.5248135225581| 621.0699999999997|1212.3848084535443| 642.0622787717556|  811.9906335826194|
|      4| 581.5248135225581| 707.3499999999999|1250.3450882615825| 642.0622787717556|  824.6440601852987|
|      5| 581.5248135225581| 621.0699999999997| 1218.194663833861| 642.0622787717556|  813.9272520427248|
|      6| 749.3654319427459| 879.9100000000003